# 평가 (Evaluation)

## 분류의 성능 평가 지표

* 정확도 (Accuracy)
* 오차행렬 (Confusion Matrix)
* 정밀도 (Precision)
* 재현율 (Recall)
* F1 스코어
* ROC AUC

## 1) 정확도 (Accuracy)

In [ ]:
'''
정확도 : 실제 데이터에서 예측 데이터가 얼마나 같은지를 판단하는 지표

                예측 결과와 동일한 데이터 건수
    정확도 = ----------------------------------
                    전체 예측 데이터 건수

       : 직관적으로 모델 예측 성능을 나타내는 지표
       : 주의 - 이거 하나만으로 성능을 평가하면 안됨 (왜곡이 발생할 수 있음)
       
    ex) 타이타닉 데이터에서 다른 조건 없이 성별 하나만 가지고 모델이 학습을 하면 여성은 생존, 남성은 사망 이런 결과가 도출

'''

In [30]:
from sklearn.base import BaseEstimator
import numpy as np

class DummyClassifier(BaseEstimator):  #괄호안 상속
    def fit(self, X, y=None):  #학습하지 않겠다
        pass
    
    def predict(self, X):  # 예측 --> 학습없는 예측, 보고 바로 판단
        pred = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
        return pred

In [31]:
# 내가 만든 알고리즘을 통해 타이타닉 생존자 예측을 다시 수행

from sklearn.preprocessing import LabelEncoder

# null처리 함수
def fillna(df):
    df['Age'].fillna(titanic_df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    return df

# 머신러닝 알고리즘에게 불필요한 속성 제거 - 사람 pk, 이름, 티켓번호
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩
def encoding_features(df):
    df['Cabin'] = df['Cabin'].str[:1]  # 앞글자 하나만 남기고 인코딩
    
    features = ['Cabin', 'Sex', 'Embarked']
    le = LabelEncoder()
    for feature in features:
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 최종 전처리 수행
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = encoding_features(df)
    return df

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

titanic_df = pd.read_csv('data/titanic_train.csv')

# 레이블 데이터 셋
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)

# 학습할 피처 데이터 셋 - 전체 전처리
X_titanic_df = transform_features(X_titanic_df)

# 학습 세트와 테스트 데이터 세트 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=11)


In [33]:
# 내가 만든 학습알고리즘 이용하여 학습/예측/평가 수행
my_clf = DummyClassifier()
my_clf.fit(X_train, y_train)

my_pred = my_clf.predict(X_test)
print('DummyClassifier: {0:.4f}'.format(accuracy_score(y_test, my_pred)))

#그냥 여자면 살고 남자면 죽게 했는데도 83%됨

DummyClassifier: 0.8324
